## Import of required Modules
Make sure, that the imported moduls are installed.

In [1]:
!pip install tqdm
!pip install numpy
!pip install ipython
!pip install pandas
!pip install selenium
!pip install newspaper3k

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will n


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [2]:
import pickle
import re
from tqdm import tqdm
import time
import numpy as np
from IPython.display import clear_output 
import pandas as pd
import time

In [3]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [4]:
from newspaper import Article

### Settings
change the settings to assert the scraper runs on your system. We reccomend to change the wait_time to a higher value if you run the scraper on a slower system or internet connection.

In [5]:
wait_time = 0.3

#### set up Selenium and Chrome Driver 
We use selenium with Chrome and tested the scraper with the chromedriver. You need the latest version of the driver from https://chromedriver.chromium.org/. Alternatively, change to the driver to a driver of your preferance.
We set up the scraper to run in the background, if you wish to run it in regular window mode, remove the line  "chrome_options.add_argument("--headless")".

In [6]:
# chromedriver setup

serv = Service(r'driver/chromedriver') #path from 'which chromedriver'

# test driver
# for headless chrome mode
chrome_options = Options()

# remove this line if you do not wish to run in background 
chrome_options.add_argument("--headless") 

### Retrieve Article Links for Crawl

In [6]:
# assemble list of links to all articles
links = []

# interact with cookie terms
chrome_path = r'driver/chromedriver' #path from 'which chromedriver'
chrome_options = Options()
chrome_options.add_argument("--headless") # open chrome in background
driver = webdriver.Chrome(service=serv, options=chrome_options)
driver.get('https://www.allsides.com/headline-roundups')
wait = WebDriverWait(driver, 10)
wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'css-47sehv')))

ele = driver.find_element(By.CLASS_NAME, "css-47sehv")
ele.click()

# retireve number of pages 
last_page_button = driver.find_element(By.CLASS_NAME, "pager-last")
link_last_page = last_page_button.find_elements(By.TAG_NAME, "a")
t = link_last_page[0].get_attribute("href")
last_page_index = int(t[-3:])

# retrieve links from start page
main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body

rows = main_table.find_elements(By.TAG_NAME, "tr")
for i in rows:
    entry = i.find_elements(By.TAG_NAME, "td")
    link = entry[0].find_element(By.TAG_NAME, "a")
    links.append(link.get_attribute("href"))
WebDriverWait(driver, 20)

# retrieve links for other pages
for page in tqdm(range(2,last_page_index+1)):   # set to max number of pages
    driver.get("https://www.allsides.com/headline-roundups?page="+str(page))
    WebDriverWait(driver, 20)
    main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body

    rows = main_table.find_elements(By.TAG_NAME, "tr")
    for i in rows:
        entry = i.find_elements(By.TAG_NAME, "td")
        link = entry[0].find_element(By.TAG_NAME, "a")
        links.append(link.get_attribute("href"))

driver.close()

 12%|█████                                     | 20/164 [01:01<07:25,  3.09s/it]


KeyboardInterrupt: 

In [18]:
# exports links as pickle file
with open("linklist_allsides_news.pickle", "wb") as f:
    pickle.dump(links, f, pickle.HIGHEST_PROTOCOL)

In [19]:
# exports links as csv
with open("linklist_allsides_news.csv", "w") as f:
    for line in links:
        print(line, file=f)

In [7]:
# functions that assert the existence of 
def check_exists_by_xpath(xpath):
    try:
        driver.find_elements(By.XPATH, xpath)
    except:
        return False
    return True

def check_exists_by_class(inp):
    try:
        driver.find_elements(By.CLASS_NAME, inp)[0]
    except:
        return False
    return True

In [8]:
# function to get all the featured political contents
def extract_political_contents():
    try: 
        list_of_contents = []
        # get the parent div
        parent_element = driver.find_element(By.XPATH, "//div[contains(@class, 'featured-coverage')]")
        #get all childs from the parents
        child_elements = parent_element.find_elements(By.XPATH, "*")
        
        # get the value of the class from each child
        for child_element in child_elements:
            class_name = child_element.get_attribute("class")
            list_of_contents.append(f"//div[contains(@class, '{class_name}')]")

        #filter list just for news item content
        filtered_list_of_contents = [item for item in list_of_contents if "news-item" in item]

        return filtered_list_of_contents
    except:
        print("path not found")
        return []

## Retrieve Articles
This section of the crawler retrieves all available news articles from AllSides along with the available information and bias tags.

In [9]:
# load link list from pickle file
with open("linklist_allsides_news.pickle", "rb") as f:
    links = pickle.load(f)

In [11]:
# list for results 
data = []

volltext = []

# retrieve information from articles in list of links
for li in tqdm(links):
    time.sleep(wait_time)
    driver = webdriver.Chrome(service=serv, options=chrome_options)
    print(li)
    # open URL
    driver.get(li)
    
    # interact with pop-up window
    if check_exists_by_class("css-47sehv"):
        ele = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((driver.find_element(By.CLASS_NAME, "css-47sehv"))))
        ele.click()
    else:
        print("no button")
    
    # netral title heading 
    try:
        heading = driver.find_element(By.TAG_NAME, "h1").text
    except:
        print("no heading found")
    
    print(heading)
    
    # date
    try:
        date = driver.find_element(By.CLASS_NAME, "date-display-single").text
    except:
        date = ""
    
    # tags
    try:
        tags = [a.text for a in driver.find_element(By.CLASS_NAME, "page-tags").find_elements(By.TAG_NAME, "a")]
    except:
        tags = ""
        
    
    # define XPATH inforamtion for article divs
    
    #divs = 
    #["/html/body/div[4]/div/div/div/div[4]/div/div/div/div[1]",
     #"/html/body/div[4]/div/div/div/div[4]/div/div/div/div[2]", 
     #"/html/body/div[4]/div/div/div/div[4]/div/div/div/div[3]"]
    
    #divs = 
    #["//div[contains(@class, 'news-item left')]", 
     #"//div[contains(@class, 'news-item center')]", 
     #"//div[contains(@class, 'news-item right')]"]
    
    divs = extract_political_contents()
    
    ##nested_div = driver.find_element_by_xpath("//div[contains(@class, 'news-item left')]")
    
    #list to store divs if they are already seen
    already_viewed_divs = []
    # access information in article divs
    for d in divs:
        if check_exists_by_xpath(d):
            try:
                #get the first found div and append to list
                div = driver.find_elements(By.XPATH,d)[0] 
                already_viewed_divs.append(div)
                #if the div is already seen take the next one
                if already_viewed_divs.count(div) > 1:
                    div = driver.find_elements(By.XPATH,d)[1]
            except:
                print("path not found")
                
            # check heading element to find out left/center/right. The title contains the bias label that we can retrieve from the text here
            try:
                cat = div.find_element(By.TAG_NAME, "h3").text
            except: 
                print("no headline found")
                
            # retrieve link to original article
            try:
                link = div.find_element(By.TAG_NAME, "a").get_attribute("href")
                print(link)
            except:
                print("no link found")
                
            # left/center/right are shuffled for each article, some roundups have e.g. only left and right articles. 
            # Thus, we have look to look each article seperately
            time.sleep(0.2)
            if "Left" in cat:
                print("left")
                try:
                    left_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading       
                except:
                    left_heading = ""
                    print("no left headline found")
                try:
                    left_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source
                except:
                    left_source = ""
                    print("no left source found")
                try:
                    left_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text body-contents
                except:
                    left_text = ""
                    print("no left text found")
                try:
                    # access link from the source
                    parent_div = div.find_element(By.CSS_SELECTOR, 'div.flex.wrap.space-between')
                    nested_a_tag = parent_div.find_element(By.CSS_SELECTOR, 'a.external-link')
                    link_href = nested_a_tag.get_attribute('href')

                    # get the fulltext from the article
                    article = Article(link_href)
                    article.download()
                    article.parse()
                    left_volltext = article.text
                    
                except:
                    left_volltext = ""
                    print("No left volltext found")
                    
                #insert code to get volltext here
                    
                # add the article information
                data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":left_heading, "source": left_source, "preview_text": left_text, "full_text": left_volltext, "bias_rating": "left"})

            elif "Right" in cat:
                print("right")
                try:
                    right_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
                except:
                    right_heading = ""
                    print("no right headline found")
                try:
                    right_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source 
                except:
                    right_source = ""
                    print("no right source found")
                try:
                    right_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
                except:
                    right_text = ""
                    print("no right text found")
                try:
                    #insert code to get volltext here
                    #link_element = div.find_element(By.CLASS_NAME, "//a[contains(@class, 'external-link')]")
                    # Extract the href attribute
                    #link_href = link_element.get_attribute("href")
                    parent_div = div.find_element(By.CSS_SELECTOR, 'div.flex.wrap.space-between')
                    nested_a_tag = parent_div.find_element(By.CSS_SELECTOR, 'a.external-link')
                    link_href = nested_a_tag.get_attribute('href')

                    # Print the extracted href
                    print("Link: ", link_href)
                    article = Article(link_href)
                    article.download()
                    article.parse()
                    right_volltext = article.text
                except:
                    right_volltext = ""
                    print("No right volltext found")
                    
                
                # add the article information
                data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading": right_heading, "source": right_source, "preview_text": right_text, "full_text": right_volltext, "bias_rating": "right"})

            else:
                print("center")
                try:
                    center_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
                except:
                    center_heading = ""
                    print("no center headline found")
                try:
                    center_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source 
                except:
                    center_source = ""
                    print("no center source found")
                try:
                    center_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
                except:
                    center_text = ""
                    print("no center text found")
                try:
                    #insert code to get volltext here
                    #link_element = div.find_element(By.CLASS_NAME, "//a[contains(@class, 'external-link')]")
                    # Extract the href attribute
                    #link_href = link_element.get_attribute("href")
                    parent_div = div.find_element(By.CSS_SELECTOR, 'div.flex.wrap.space-between')
                    nested_a_tag = parent_div.find_element(By.CSS_SELECTOR, 'a.external-link')
                    link_href = nested_a_tag.get_attribute('href')

                    # Print the extracted href
                    print("Link: ", link_href)
                    article = Article(link_href)
                    article.download()
                    article.parse()
                    center_volltext = article.text
                except:
                    center_volltext = ""
                    print("No center volltext found")
                    
                # add the article information
                data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading": center_heading, "source": center_source, "preview_text": center_text, "full_text": center_volltext, "bias_rating": "center"})
        else:
            print("div not found")
            
    
    
    
    driver.close()
    
    # clear output
    #clear_output()
    # added a wait here to assert the scraper runs well
    time.sleep(wait_time)


  0%|                                                  | 0/2600 [00:00<?, ?it/s]

https://www.allsides.com/story/supreme-court-justice-alito-responds-propublica-inquiry-wall-street-journal-op-ed
Justice Alito Responds to ProPublica Inquiry With Wall Street Journal Op-ed
https://www.allsides.com/news/2023-06-21-0639/supreme-court-justice-samuel-alito-took-luxury-fishing-vacation-gop-billionaire
left
Link:  https://www.propublica.org/article/samuel-alito-luxury-fishing-trip-paul-singer-scotus-supreme-court
https://www.allsides.com/news/2023-06-21-0637/supreme-court-justice-alito-reveals-trip-billionaire-paul-singer-op-ed-and
center
Link:  https://www.forbes.com/sites/antoniopequenoiv/2023/06/20/justice-alito-reveals-trip-with-billionaire-paul-singer-in-op-ed-and-defends-himself/?sh=6b1d4eb29bd5
No center volltext found
https://www.allsides.com/news/2023-06-21-0636/supreme-court-justice-samuel-alito-propublica-misleads-its-readers
right
Link:  https://www.wsj.com/articles/propublica-misleads-its-readers-alito-gifts-disclosure-alaska-singer-23b51eda
No right volltext fo


  0%|                                       | 1/2600 [00:16<11:34:57, 16.04s/it]

https://www.allsides.com/story/politics-lawmakers-2024-candidates-react-hunter-biden-plea-deal
Lawmakers, 2024 Candidates React to Hunter Biden Plea Deal
https://www.allsides.com/news/2023-06-20-1526/politics-hunter-biden-plea-deal-sparks-awkward-silence-democrats
center
Link:  https://www.newsweek.com/hunter-biden-plea-deal-sparks-awkward-silence-democrats-1807961
No center volltext found
https://www.allsides.com/news/2023-06-20-1528/politics-intentionally-provocative-hunter-biden-plea-deal
right
Link:  https://www.nationalreview.com/corner/the-intentionally-provocative-hunter-biden-plea-deal/
https://www.allsides.com/news/2023-06-20-1524/politics-hunter-biden-story-far-over
left
Link:  https://www.cnn.com/2023/06/20/politics/charges-against-hunter-biden-what-matters/index.html



  0%|                                        | 2/2600 [00:28<9:52:29, 13.68s/it]

...


100%|████████████████████████████████████▉| 2598/2600 [9:01:59<00:32, 16.20s/it]

https://www.allsides.com/story/supreme-court-lifts-california-restrictions-home-worship-gatherings
Supreme Court Lifts California Restrictions On At-Home Worship Gatherings
https://www.allsides.com/news/2021-04-11-1436/supreme-court-rules-worshippers-and-against-california-covid-restrictions
left
Link:  https://www.npr.org/2021/04/10/986010977/supreme-court-rules-for-worshippers-and-against-california-covid-restrictions
https://www.allsides.com/news/2021-04-11-1432/supreme-court-strikes-another-pandemic-related-restriction-religious-services
left
Link:  https://www.washingtonpost.com/politics/courts_law/supreme-court-california-coronavirus-church/2021/04/10/182faec2-99e8-11eb-962b-78c1d8228819_story.html
https://www.allsides.com/news/2021-04-11-1430/supreme-court-rules-against-californias-limits-home-religious-gatherings
right
Link:  https://www.foxnews.com/politics/supreme-court-rules-against-californias-limits-on-in-home-relgious-gatherings



100%|████████████████████████████████████▉| 2599/2600 [9:02:13<00:15, 15.52s/it]

https://www.allsides.com/story/matt-gaetz-speaks-out-against-sexual-misconduct-allegations
Matt Gaetz Speaks Out Against Sexual Misconduct Allegations
https://www.allsides.com/news/2021-04-10-1900/matt-gaetz-tom-reed-being-investigated-house-ethics-committee-sexual-misconduct
center
Link:  https://www.newsweek.com/matt-gaetz-tom-reed-being-investigated-house-ethics-committee-sexual-misconduct-allegations-1582539
No center volltext found
https://www.allsides.com/news/2021-04-10-1857/matt-gaetz-shows-donald-trumps-defiant-style-here-stay
left
Link:  https://www.cnn.com/2021/04/10/politics/matt-gaetz-donald-trump-republicans/index.html
https://www.allsides.com/news/2021-04-10-1855/defiant-matt-gaetz-vows-truth-will-prevail-fiery-speech-amid-doj-investigation
right
Link:  https://www.washingtonexaminer.com/news/gaetz-defiant-of-victory-amid-doj-investigation


100%|█████████████████████████████████████| 2600/2600 [9:02:25<00:00, 12.52s/it]


In [12]:
## convert data to dataframe
df = pd.DataFrame(data)

# export scraped articles
df.to_csv("allsides_news_complete.csv")
print(volltext)

[]
